# 🤖 AI, CA3, Machine Learning 📚  

* **Name** : Mohammad Sadra 🖊  
* **Last Name** :  Abbasi 📝  
* **SID** : 810101469 🆔

# IMPORT


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


# Preprocess

In [2]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, target_col='crisis_type', task_type='classification', 
                 numerical_strategy='mean', categorical_strategy='most_frequent'):
        """
        task_type: 'classification' (default) -> encodes target labels
                   'regression' -> keeps target as continuous numbers
        """
        self.target_col = target_col
        self.task_type = task_type 
        self.num_strategy = numerical_strategy
        self.cat_strategy = categorical_strategy
        
        self.num_imputer = None
        self.cat_imputer = None
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.target_encoder = LabelEncoder()
        
        self.num_cols = []
        self.cat_cols = []
        self.outlier_bounds = {}

    def _feature_engineering(self, df):
        df_copy = df.copy()
        if 'avg_sleep_last_week' in df_copy.columns and 'sleep_hours_before_exam' in df_copy.columns:
            df_copy['sleep_stability'] = abs(df_copy['avg_sleep_last_week'] - df_copy['sleep_hours_before_exam'])
        return df_copy

    def fit(self, X, y=None):
        X = self._feature_engineering(X)
        
        self.num_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
        self.cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
        
        if self.target_col in self.num_cols: self.num_cols.remove(self.target_col)
        if self.target_col in self.cat_cols: self.cat_cols.remove(self.target_col)
        
        self.num_imputer = SimpleImputer(strategy=self.num_strategy)
        self.num_imputer.fit(X[self.num_cols])
        
        self.cat_imputer = SimpleImputer(strategy=self.cat_strategy)
        self.cat_imputer.fit(X[self.cat_cols])
        
        for col in self.num_cols:
            Q1 = X[col].quantile(0.01)
            Q3 = X[col].quantile(0.99)
            self.outlier_bounds[col] = (Q1, Q3)

        for col in self.cat_cols:
            le = LabelEncoder()
            le.fit(X[col].astype(str))
            self.label_encoders[col] = le
            
        if y is not None and self.task_type == 'classification':
            self.target_encoder.fit(y)
            
        X_temp_num = self.num_imputer.transform(X[self.num_cols])
        self.scaler.fit(X_temp_num)
        
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X = self._feature_engineering(X)
        
        X[self.num_cols] = self.num_imputer.transform(X[self.num_cols])
        X[self.cat_cols] = self.cat_imputer.transform(X[self.cat_cols])
        
        for col in self.num_cols:
            lower, upper = self.outlier_bounds[col]
            X[col] = np.clip(X[col], lower, upper)
            
        for col in self.cat_cols:
            le = self.label_encoders[col]
            X[col] = X[col].astype(str).map(lambda s: s if s in le.classes_ else le.classes_[0])
            X[col] = le.transform(X[col])
            
        X[self.num_cols] = self.scaler.transform(X[self.num_cols].values)
        
        X_final = np.hstack([X[self.num_cols].values, X[self.cat_cols].values])
        
        y_transformed = None
        if y is not None:
            if self.task_type == 'classification':
                y_transformed = self.target_encoder.transform(y)
            else:
                y_transformed = y.values if isinstance(y, pd.Series) else y
            
        if y is not None:
            return X_final, y_transformed
        else:
            return X_final
        
    def plot_correlation(self, X, y=None):
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        df_temp = pd.DataFrame(X, columns=self.num_cols + self.cat_cols)
        
        if y is not None:
            df_temp[self.target_col] = y
            
        plt.figure(figsize=(12, 10))
        sns.heatmap(df_temp.corr(), annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
        plt.title("Feature Correlation Matrix")
        plt.show()

In [3]:
df = pd.read_csv('./part4_dataset/part4_dataset.csv')
target_col = 'final_exam_score'

X = df.drop(columns=[target_col]) 
y = df[target_col]

X_train_raw, X_val_raw, y_train_raw, y_val_raw = train_test_split(X, y, test_size=0.3, random_state=42)

preprocessor = Preprocessor(target_col=target_col, task_type='regression')

preprocessor.fit(X_train_raw, y_train_raw)

X_train, y_train = preprocessor.transform(X_train_raw, y_train_raw)
X_val, y_val = preprocessor.transform(X_val_raw, y_val_raw)

# preprocessor.plot_correlation(X_train, y_train_transformed)

# Metric implementation

In [4]:
def calc_mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def calc_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def calc_r2(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    if ss_tot == 0: return 0
    return 1 - (ss_res / ss_tot)

##

<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">توضیح متریک ها</h3>
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
به طور کلی در این بخش با یک متغیر هدف پیوسته سروکار داریم و برخلاف بخش‌های قبلی، پیش‌بینی دقیق یک مقدار مشخص به عنوان لیبل عملا امکان‌پذیر نیست. در چنین مسائلی، هدف اصلی این نیست که مقدار پیش‌بینی‌شده دقیقا با مقدار واقعی برابر باشد، بلکه میزان نزدیکی پیش‌بینی به مقدار واقعی اهمیت دارد. به همین دلیل، از معیارهایی استفاده می‌شود که نشان دهند مدل تا چه حد به مقدار صحیح نزدیک شده است.
</p>

<p>
<strong>MSE</strong> یا Mean Squared Error برابر با میانگین توان دوم اختلاف بین مقدار واقعی و مقدار پیش‌بینی‌شده است. به دلیل استفاده از توان دوم، این معیار به خطاهای بزرگ حساس‌تر است و آن‌ها را بیش از خطاهای کوچک جریمه می‌کند. در نتیجه، MSE بیشتر زمانی اهمیت دارد که خطاهای بزرگ برای مسئله نامطلوب باشند.
</p>

<p>
<strong>MAE</strong> یا Mean Absolute Error مشخص می‌کند که مدل به طور میانگین چه میزان از مقدار واقعی فاصله دارد. این معیار تفسیر ساده‌تری نسبت به MSE دارد و به صورت مستقیم میزان خطای متوسط مدل را در همان واحد متغیر هدف نشان می‌دهد.
</p>

<p>
<strong>R<sup>2</sup></strong> یا ضریب تعیین نشان می‌دهد که چه درصدی از تغییرات متغیر هدف توسط مدل توضیح داده شده است. مقدار این معیار بیانگر میزان تطابق پیش‌بینی‌های مدل با داده‌های واقعی نسبت به یک مدل ساده مرجع (مانند پیش‌بینی میانگین) است. هرچه مقدار R<sup>2</sup> به ۱ نزدیک‌تر باشد، مدل توانسته است الگوی کلی داده‌ها را بهتر یاد بگیرد، و مقادیر پایین یا منفی نشان‌دهنده عملکرد ضعیف مدل هستند.
</p>

</div>




# Regression Model Evaluation

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

regression_models = {
    "Decision Tree": DecisionTreeRegressor(max_depth=5, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
}

results_log = []

for model_name, regressor in regression_models.items():
    regressor.fit(X_train, y_train)
    
    y_pred_val = regressor.predict(X_val)
    
    mse = calc_mse(y_val, y_pred_val)
    mae = calc_mae(y_val, y_pred_val)
    r2 = calc_r2(y_val, y_pred_val)
    
    results_log.append({'Model': model_name, 'R2': r2, 'MSE': mse})
    
    print(f"Model: {model_name}")
    print(f"MSE: {mse:.4f} | MAE: {mae:.4f} | R2: {r2:.4f}")
    print("=" * 40)

Model: Decision Tree
MSE: 7.0817 | MAE: 1.9918 | R2: 0.6400
Model: Random Forest
MSE: 6.1310 | MAE: 1.7379 | R2: 0.6883
Model: XGBoost
MSE: 6.2926 | MAE: 1.7532 | R2: 0.6801


### result

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/regression_models_result.png" width="600"></td>
    </tr>
</table>



# Bias–Variance Experiment

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

class BiasVarianceAnalyzer:
    def __init__(self, X_train, y_train, X_val, y_val):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        
        self.x_star = None
        self.y_star_actual = None
        self.predictions = {
            "Decision Tree": [],
            "Random Forest": [],
            "XGBoost": []
        }

    def set_target_point(self, index=0):
        if isinstance(self.X_val, pd.DataFrame):
            self.x_star = self.X_val.iloc[index : index+1]
            self.y_star_actual = self.y_val.iloc[index]
        else:
            self.x_star = self.X_val[index].reshape(1, -1)
            self.y_star_actual = self.y_val[index]
            
    def run_experiment(self, n_iterations=100, train_ratio=0.3):
        self.predictions = {k: [] for k in self.predictions}
        
        for i in range(n_iterations):
            X_sub, _, y_sub, _ = train_test_split(
                self.X_train, self.y_train, 
                train_size=train_ratio, 
                random_state=i
            )
            
            models = {
                "Decision Tree": DecisionTreeRegressor(max_depth=5, random_state=42),
                "Random Forest": RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42),
                "XGBoost": XGBRegressor(n_estimators=50, learning_rate=0.1, max_depth=5, random_state=42)
            }
            
            for name, model in models.items():
                model.fit(X_sub, y_sub)
                pred = model.predict(self.x_star)[0]
                self.predictions[name].append(pred)
            
        print("Experiment completed.")

    def plot_bias_variance(self):
        plt.figure(figsize=(10, 6))
        
        # define the positions of the models
        positions = {
            "Decision Tree": 0,
            "Random Forest": 1,
            "XGBoost": 2
        }
        
        colors = {'Decision Tree': 'red', 'Random Forest': 'blue', 'XGBoost': 'green'}
        
        for name, preds in self.predictions.items():
            x_pos = positions[name]
            y_vals = preds
            
            x_vals = np.full(len(y_vals), x_pos)
            
            plt.scatter(x_vals, y_vals, color=colors[name], alpha=0.3, s=30, label=name)
            
            mean_pred = np.mean(y_vals)
            plt.scatter(x_pos, mean_pred, color='black', marker='X', s=100, zorder=10)

        plt.axhline(y=self.y_star_actual, color='gray', linestyle='--', label='Actual Value')
        
        plt.xticks([0, 1, 2], positions.keys())
        plt.ylabel("Predicted Value for x_star")
        plt.title("Bias-Variance Tradeoff: Model Stability Analysis")
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        
        plt.show()

analyzer = BiasVarianceAnalyzer(X_train, y_train, X_val, y_val)
analyzer.set_target_point(index=5)
analyzer.run_experiment(n_iterations=100, train_ratio=0.3)
analyzer.plot_bias_variance()

##

<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">خروجی</h3>
</div>

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/model_analysis.png" width="800"></td>
    </tr>
</table>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
می‌توان مشاهده کرد که مدل درخت تصمیم واریانس بالاتری دارد؛ به این معنا که مقادیر پیش‌بینی‌شده آن روی خط افقی پراکندگی زیادی دارند و تراکم نقاط پایین‌تر است. این رفتار نشان می‌دهد که با هر بار انتخاب تصادفی داده‌های آموزشی، خروجی مدل به شکل محسوسی تغییر می‌کند و مدل از پایداری کمتری برخوردار است. در مقابل، دو مدل Random Forest و XGBoost تراکم بالاتری از پیش‌بینی‌ها را در اطراف یک مقدار مشخص نشان می‌دهند که بیانگر واریانس کمتر و پایداری بیشتر آن‌ها نسبت به درخت تصمیم است.

در مورد بایاس، هرچه میانگین پیش‌بینی‌ها که در نمودار با علامت ضربدر سیاه مشخص شده است به مقدار واقعی نزدیک‌تر باشد، بایاس مدل کمتر خواهد بود. بر اساس این معیار، مشاهده می‌شود که مدل Random Forest میانگینی نزدیک‌تر به مقدار واقعی دارد و در نتیجه نسبت به دو مدل دیگر از بایاس کمتری برخوردار است، در حالی که مدل درخت تصمیم و XGBoost فاصله بیشتری با مقدار حقیقی نشان می‌دهند.
</p>

</div>


# runner

In [6]:
import pandas as pd
import numpy as np
import os

test_file = "./test/part4_test.csv" 
output_folder = "part4_submissions"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

try:
    df_test = pd.read_csv(test_file)
    print("Test data loaded successfully.")
except FileNotFoundError:
    print("Warning: Test file not found!")
    df_test = pd.DataFrame(columns=X_train.columns)

X_test_transformed = preprocessor.transform(df_test)

# train models
dt = DecisionTreeRegressor(max_depth=5, random_state=42)
rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
xgb = XGBRegressor(n_estimators=50, learning_rate=0.1, max_depth=5, random_state=42)

dt.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)


def generate_submission(model, X, original_df, filename, model_name):
    print(f"Generating results for: {model_name}...")
    preds = model.predict(X)
    submission = original_df.copy()
    submission['label'] = preds
    save_path = os.path.join(output_folder, filename)
    submission.to_csv(save_path, index=False)
    print(f"Saved: {save_path}")

generate_submission(
    model= dt,
    X=X_test_transformed, 
    original_df=df_test,
    filename="DecisionTree_Part4.csv",
    model_name="Decision Tree"
)


generate_submission(
    model=rf, 
    X=X_test_transformed, 
    original_df=df_test,
    filename="RandomForest_Part4.csv",
    model_name="Random Forest"
)

generate_submission(
    model=xgb, 
    X=X_test_transformed, 
    original_df=df_test,
    filename="XGBoost_Part4.csv",
    model_name="XGBoost"
)

print("\n" + "="*50)
print("All Part 4 submissions generated successfully!")
print("="*50)

Test data loaded successfully.
Generating results for: Decision Tree...
Saved: part4_submissions/DecisionTree_Part4.csv
Generating results for: Random Forest...
Saved: part4_submissions/RandomForest_Part4.csv
Generating results for: XGBoost...
Saved: part4_submissions/XGBoost_Part4.csv

All Part 4 submissions generated successfully!
